### Importing

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, ArtistAnimation, Animation
from matplotlib.lines import Line2D
import matplotlib

matplotlib.use('TkAgg')  # Use an appropriate backend (e.g., TkAgg)
import matplotlib.patheffects as path_effects
import matplotlib.patches as patches

In [2]:
# Specify the relative path to the data directory
non_games_data_folder_path = "C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Non_Games_Data"

# List all files in the data folder
file_list_non_games = os.listdir(non_games_data_folder_path)

# Use glob to filter specific file types
csv_files_non_games = glob.glob(os.path.join(non_games_data_folder_path, "*.csv"))

# Read in the supplementary data
games = pd.read_csv(csv_files_non_games[0])
nfl_colors = pd.read_csv(csv_files_non_games[1])
players = pd.read_csv(csv_files_non_games[2])
plays = pd.read_csv(csv_files_non_games[3])
clean_df = pd.read_csv("C:/Users/sethl/OneDrive/Important Stuff/R/R files/NFL/DataBowl/2024-Big-Data-Bowl/Created_DF/clean_df.csv")

### Validation Viz

In [7]:
gameId = 2022091110
playId = 171

In [8]:
# Select specific columns from 'plays'
selected_plays = plays[["gameId", "playId", "playDescription"]]

# Filter rows based on conditions
filtered_plays = selected_plays[
    (selected_plays["gameId"] == gameId) & (selected_plays["playId"] == playId)
]

# Perform left join with 'df'
VizDF = filtered_plays.merge(clean_df, how="left")

VizDF["x_temp"] = VizDF["y"]
VizDF["y_temp"] = VizDF["x"]
VizDF["y"] = VizDF["y_temp"]
VizDF["x"] = VizDF["x_temp"]
VizDF = VizDF.drop(["x_temp", "y_temp"], axis=1)
VizDF["jerseyNumber"] = np.where(VizDF["jerseyNumber"].isna(), 0, VizDF["jerseyNumber"])

ymin = 0
ymax = 160 / 3
hash_right = 38.35
hash_left = 12
hash_width = 3.3
xmin = max(round(min(VizDF["y"]) - 10, -1), 0)
xmax = min(round(max(VizDF["y"]) + 10, -1), 120)

# Create a grid of coordinates (x, y)
y_values = [0, 23.36667, 29.96667, ymax]
x_values = list(range(10, 111))  # 10 to 110
grid = pd.DataFrame({"x": x_values * len(y_values), "y": y_values * len(x_values)})

# Filter rows based on conditions
df_hash = grid[(grid["x"] % 5 != 0) & (grid["x"] < xmax) & (grid["x"] > xmin)]
# Create the base plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

for index, row in df_hash.iterrows():
    if row["y"] > 55 / 2:
        ax.annotate(
            "|", xy=(row["x"] - 0.25, row["y"]), va="top", color="gray", zorder=0
        )
    else:
        ax.annotate(
            "|", xy=(row["x"] - 0.25, row["y"]), va="bottom", color="gray", zorder=0
        )

# Add segment annotations
for x in range(int(max(9, xmin)), int(min(xmax, 111)), 5):
    ax.plot([x, x], [ymin, ymax], color="gray", zorder=0)


# Add text annotations on the sides
y_labels = (
    ["   G"]
    + [str(val) for val in range(10, 51, 10)]
    + [str(val) for val in reversed(range(10, 41, 10))]
    + ["G   "]
)

yard_labels = pd.DataFrame(zip(range(10, 120, 10), y_labels), columns=["yard", "label"])

for x in range(int(max(10, xmin)), int(min(xmax, 120)), 10):
    if (x != xmin) & (x != xmax):
        label_top = yard_labels.loc[yard_labels["yard"] == x, "label"].values[0]
        label_bottom = yard_labels.loc[yard_labels["yard"] == x, "label"].values[0]

        ax.text(
            x,
            hash_left - 2,
            label_top,
            size=12,
            color="gray",
            va="center",
            ha="center",
            zorder=0,
        )
        ax.text(
            x,
            ymax - hash_left + 2,
            label_bottom,
            size=12,
            color="gray",
            va="center",
            ha="center",
            zorder=0,
        )

# Add field boundary lines
ax.plot(
    [xmin, xmin, xmax, xmax, xmin],
    [ymin, ymax, ymax, ymin, ymin],
    color="gray",
    zorder=0,
)

# Turn off both axes and tick labels
ax.axis("off")

# Initialize the artists
point_artists = []
annotation_artists = []

def init():
    # Initialize the artists (empty at the beginning)
    return point_artists + annotation_artists


def update(frame):
    # Update the plot for each frame

    # Clear the artists from the previous frame
    for artist in point_artists + annotation_artists:
        artist.remove()
    point_artists.clear()
    annotation_artists.clear()

    # Get the rows for the current frame from the DataFrame
    frame_rows = VizDF[VizDF["frameId"] == frame + 5]

    # Add/update dots for this frame
    for index, row in frame_rows.iterrows():
        x = row["y"]
        y = row["x"]
        color = row["primary"]
        jersey = row["jerseyNumber"]
        team = row["club"]

        if team == "football":
            dot_artist1 = Line2D([x], [y], marker="d", markersize=8, color=color)
            ax.add_artist(dot_artist1)
            point_artists.append(dot_artist1)

        else:
            dot_artist = Line2D(
                [x], [y], marker="o", markersize=20, color=color, zorder=2
            )
            ax.add_artist(dot_artist)
            point_artists.append(dot_artist)

            annotation_artist = ax.text(
                x,
                y,
                str(round(int(jersey), 0)),
                fontsize=10,
                ha="center",
                va="center",
                color="white",
                zorder=3,
                path_effects=[
                    path_effects.Stroke(linewidth=0.5, foreground="black"),
                    path_effects.Normal(),
                ],
            )
            annotation_artists.append(annotation_artist)


    return annotation_artists + point_artists


ex_play_lengthVIZ = VizDF["frameId"].nunique()

# Create the animation
animation = FuncAnimation(
    fig,
    update,
    frames=range(ex_play_lengthVIZ),
    init_func=init,
    blit=True,
    interval=100,
)

#animation.save("Sample Validation Viz.gif")
plt.show()